In [1]:
import os
from openai import OpenAI

if not os.environ["OPENAI_API_KEY"]: 
    os.environ["OPENAI_API_KEY"]= '<REDACTED>'

MODEL = "gpt-4o-mini"

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [2]:
import json

def read_jsonl_and_transform(file_path):
    result_dict = {}

    with open(file_path, 'r') as file:
        for line in file:
            record = json.loads(line)
            keyword = record.pop('keyword')
            result_dict[keyword] = record

    return result_dict

def read_json_to_dict(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

keyword_labels, results = read_jsonl_and_transform('20qs-data/labeled_keywords.jsonl'), read_json_to_dict('20qs-data/labeling_results.jsonl')

In [5]:
things_list = [word for word in keyword_labels if keyword_labels[word]['place'] == 'no']

In [19]:
def make_similar_keyword(keyword):
    
    sys_prompt = ("You are an assistant tasked with generating a keyword similar to a given keyword for a game of 20 Questions. "
                  "The similar keyword should belong to the same general category but must not be a synonym or extremely similar. "
                  "Here are some guiding examples:\n"
                  "Example 1:\n Keyword: Mechanical pencil\n Similar Keyword: Ballpoint pen\n"
                  "Example 2:\n Keyword: Surgical mask\n Similar Keyword: Forceps\n"
                  "Example 3:\n Keyword: Honda Civic\n Similar Keyword: Toyota Highlander\n"
                  "Example 4:\n Keyword: Chocolate Mousse\n Similar Keyword: Vanilla cake\n"
                  "Only output the similar keyword and nothing else.\n")
    user_msg = f"Keyword:{keyword}\nSimilar Keyword:"
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": sys_prompt},
            {"role": "user", "content": user_msg}
        ],
        max_tokens=10
    )
    response = response.choices[0].message.content

    return response

In [21]:
from tqdm import tqdm

similar_keywords = []
for keyword in tqdm(things_list, desc = 'keywords generated'):
    similar_keyword = make_similar_keyword(keyword)
    similar_keywords.append(similar_keyword)    

keywords generated: 100%|██████████| 1327/1327 [11:13<00:00,  1.97it/s]


In [22]:
with open('20qs-data/similar_keywords.txt', 'w') as file:
    for item in similar_keywords:
        file.write(f"{item}\n")